###Ingest qualifying json files

In [0]:
dbutils.widgets.text('p_data_source','')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#####Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [0]:
qualifying_schema = StructType(fields = [StructField('qualifyId',IntegerType(),False),
                                        StructField('raceId',IntegerType(),True),
                                        StructField('driverId',IntegerType(),True),
                                        StructField('constructorId',IntegerType(),True),
                                        StructField('number',IntegerType(),True),
                                        StructField('position',IntegerType(),True),
                                        StructField('q1',StringType(),True),
                                        StructField('q2',StringType(),True),
                                        StructField('q3',StringType(),True)])

In [0]:
qualifying_df = spark.read.option('multiline',True).json(f"{raw_folder_path}/qualifying",schema = qualifying_schema)

#####Step 2 - Rename columns and add new columns

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
qualifying_final_df = qualifying_df.withColumnRenamed('qualifyId','qualify_id') \
        .withColumnRenamed('driverId','driver_id') \
        .withColumnRenamed('raceId','race_id') \
        .withColumnRenamed('constructorId','constructor_id') \
        .withColumn('ingestion_date',current_timestamp()) \
        .withColumn('data_source',lit(v_data_source))

#####Step 3 - Write the output to processed container in parquet format

In [0]:
qualifying_final_df.write.mode('overwrite').parquet(f"{processed_folder_path}/qualifying")

In [0]:
dbutils.notebook.exit('Success')